In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import re
import torch

import os
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ====================================================
# Data Loading
# ====================================================
INPUT_DIR = '../input/us-patent-phrase-to-phrase-matching/'
OUTPUT_DIR = './'
train = pd.read_csv(INPUT_DIR+'train.csv')
test = pd.read_csv(INPUT_DIR+'test.csv')
submission = pd.read_csv(INPUT_DIR+'sample_submission.csv')
print(f"train.shape: {train.shape}")
print(f"test.shape: {test.shape}")
print(f"submission.shape: {submission.shape}")
display(train.head())
display(test.head())
display(submission.head())

In [ ]:
# !pip install --upgrade python

# import pandas as pd

In [ ]:
import pickle

df = pd.read_pickle('../input/electra-base-find-embeds/oof_df.pkl')
df

In [ ]:
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score

labels = df['score'].values
preds = df['pred'].values
score = get_score(labels, preds)
print(f'Before PP OOF: {score}')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk import pos_tag
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
import pickle
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# stack_df = df[['id','score','fold','pred']].copy()
stack_df = df.copy()

In [ ]:
stack_df.head(3)

In [ ]:
stop_words = stopwords.words('english')


In [ ]:
def future(df):
    df['cate'] = df['context'].apply(lambda x: x[0])
    df['anchor_len'] = df['anchor'].apply(lambda x: len(x))
    df['target_len'] = df['target'].apply(lambda x: len(x))
    return df

In [ ]:
import numpy as np

def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
#     print (matrix)
    return (matrix[size_x - 1, size_y - 1])


In [ ]:
stack_df['lev_dist'] = stack_df.apply(lambda x: levenshtein(x.anchor, x.target), axis=1)

In [ ]:
stack_df = future(stack_df)

In [ ]:
# tr_ts=['context','anchor_token','target_token','target_len','target_nostop','anchor_nostop']
tr_ts=['anchor_len','target_len','lev_dist']#,'target_nostop','anchor_nostop']


In [ ]:
# stack_df[tr_ts].to_csv('data_sessions_text.txt', 
#                                  sep=' ', index=None, header=None)

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# cv = TfidfVectorizer(ngram_range=(1,3), max_features=3)
# with open('data_sessions_text.txt') as inp_file:
#     data = cv.fit_transform(inp_file)

In [ ]:
stack_df.shape

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [ ]:
num = 200

In [ ]:
# KMeans()

In [ ]:
def get_valid_df(fold,stack_df):
    val_df = stack_df[stack_df['fold']==fold]
    val_feats = np.load(f'../input/electra-base-find-embeds/embeds_fold{fold}.npy')
#     val_feats = KMeans(n_clusters=num,random_state=69).fit_transform(val_feats)   
    val_feats = PCA(n_components=num,random_state=69).fit_transform(val_feats)   

    val_df[[f'emb_{i}' for i in range(0,num)]] = val_feats
    return val_feats, val_df


def get_train_df(fold,stack_df):
    train_df = pd.DataFrame(columns = list(stack_df.columns))
    train_feats = []
    for i in range(4):
        if i == fold:
            continue
        else:
            val_df = stack_df[stack_df['fold']==i]
            val_feats = np.load(f'../input/electra-base-find-embeds/embeds_fold{i}.npy')
#             val_feats = KMeans(n_clusters=num,random_state=69).fit_transform(val_feats)    
            val_feats = PCA(n_components=num,random_state=69).fit_transform(val_feats)   

            val_df[[f'emb_{i}' for i in range(0,num)]] = val_feats
            
            train_df = pd.concat([train_df,val_df],axis=0)
            train_feats.append(val_feats)
    train_feats = np.concatenate(train_feats)
        
    return train_feats, train_df

In [ ]:
# stack_df


In [ ]:
from sklearn.metrics import *

In [ ]:
import xgboost as xgb
xgb_params = {'learning_rate': 0.09827605967564293,
              'reg_lambda': 7.960622217848342e-07, 
              'subsample': 0.7422597612762745,
              'max_depth': 4, 'early_stopping_rounds': 353,
              'n_estimators': 10000,'cat_features': ['cate'] }

xgb_params2 = {'learning_rate': 0.09827605967564293,'tree_method':'gpu_hist', 'gpu_id':0,
                 'early_stopping_rounds': 50,
              'n_estimators': 10000, }

In [ ]:
class CatBoostEvalMetricPearson(object):
    def get_final_error(self, error, weight):
        return error

    def is_max_optimal(self):
        # the larger metric value the better
        return True

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        preds = np.array(approxes[0])
        target = np.array(target)
        return np.corrcoef(target, preds)[0, 1], 0

In [ ]:
tr_ts=['pred','anchor_len','target_len','lev_dist','cate']#,'target_nostop','anchor_nostop']
tr_ts += [f'emb_{i}' for i in range(0,num)]

In [ ]:
# from cuml.svm import SVC
import gc
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor, Pool
from scipy import sparse

# from cuml.linear_model import LogisticRegression
import lightgbm as lgb
oofs = []
for fold in range(4):
    
    print("Training Fold",fold)
    
    X,train = get_train_df(fold,stack_df)
    x_valid,val = get_valid_df(fold,stack_df)
    
    X2 = train[tr_ts]#.values#.reshape(-1,1)
    x_valid2 = val[tr_ts]#.values#.reshape(-1,1)
#     print(train.fold.value_counts())
#     print(val.fold.value_counts())
    

    X = X2
    x_valid = x_valid2
#     print(X.shape)

#     X = np.concatenate((X,X2),axis=1)
#     x_valid = np.concatenate((x_valid,x_valid2),axis=1)
    
#     /X = sparse.csr_matrix(X)
#     x_valid = sparse.csr_matrix(x_valid)
    
    
    y = train['score']
    y_valid = val['score']
    
    
    rmse_before = mean_squared_error(val['score'],val['pred'],squared=False)
    print(f'Before Rmse val score: {rmse_before} \n')
    print('*******************************************')

#     clf2 = xgb.XGBRegressor(**xgb_params,objective=pearson_cumsom_loss)
    clf2 = CatBoostRegressor(**xgb_params,

            eval_metric=CatBoostEvalMetricPearson()
                             
        )
    clf2.fit(X, y,
                  eval_set=[(X, y), (x_valid, y_valid)],
                  verbose=50,
#                   categorical_feature=cate_ft_lst,
                  )  
    

    save_to = '{}clf2_fold{}.dat'.format('/kaggle/working/', fold)
    pickle.dump(clf2, open(save_to, "wb"))
    
    clf2 = pickle.load(open('{}clf2_fold{}.dat'.format('/kaggle/working/', fold),'rb'))
    val['conf'] = clf2.predict(x_valid)#[:,1]  

    
    sc_before = get_score(val['score'],val['pred'])
    sc_after = get_score(val['score'],val['conf'])
    
    print(f'Before PP val score: {sc_before} \n')
    print('*******************************************')
    print(f'After PP val score: {sc_after} \n')
    print('*******************************************')
    print('*******************************************')
    print(f'Fold {fold} finished.....')
    
    
    val['fold'] = fold
    oofs.append(val)
    
    del train, val, X, y, x_valid, y_valid, clf2
    gc.collect()
    
oofs= pd.concat(oofs).reset_index(drop=True)

In [ ]:
a = get_score(oofs['score'],oofs['pred'])
b = get_score(oofs['score'],oofs['conf'])

print(f'OOF Before PP: {a}')
print(f'OOF After PP: {b}')